In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

2022-03-30 09:26:13.256230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 09:26:13.289928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-30 09:26:13.289944: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-30 09:26:13.290516: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [3]:
#k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

In [4]:
paths = ['/home/wander/OtherProjects/har_flower/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            'neurons_1' : 32, 
            'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    test_results, ba = har.run()
    #model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
print(max(bas))


Epoch 1/40
254/254 [==============================] - 1s 3ms/step - loss: 0.1482 - avg_multilabel_BA_2: 0.8635 - val_loss: 0.1388 - val_avg_multilabel_BA_2: 0.9029
Epoch 2/40
254/254 [==============================] - 0s 1ms/step - loss: 0.1305 - avg_multilabel_BA_2: 0.9051 - val_loss: 0.1507 - val_avg_multilabel_BA_2: 0.9062
Epoch 3/40
254/254 [==============================] - 0s 1ms/step - loss: 0.1435 - avg_multilabel_BA_2: 0.9088 - val_loss: 0.2032 - val_avg_multilabel_BA_2: 0.9094
Epoch 4/40
254/254 [==============================] - 0s 1ms/step - loss: 0.1448 - avg_multilabel_BA_2: 0.9088 - val_loss: 0.1791 - val_avg_multilabel_BA_2: 0.9083
Epoch 5/40
254/254 [==============================] - 0s 1ms/step - loss: 0.1389 - avg_multilabel_BA_2: 0.9066 - val_loss: 0.1618 - val_avg_multilabel_BA_2: 0.9076
Epoch 6/40
254/254 [==============================] - 0s 1ms/step - loss: 0.1374 - avg_multilabel_BA_2: 0.9077 - val_loss: 0.1394 - val_avg_multilabel_BA_2: 0.9083
Epoch 7/40
254/2

199/199 [==============================] - 0s 1ms/step - loss: 0.2505 - avg_multilabel_BA_2: 0.8783 - val_loss: 0.2419 - val_avg_multilabel_BA_2: 0.8766
Epoch 10/40
199/199 [==============================] - 0s 1ms/step - loss: 0.2518 - avg_multilabel_BA_2: 0.8752 - val_loss: 0.2421 - val_avg_multilabel_BA_2: 0.8734
Epoch 11/40
199/199 [==============================] - 0s 1ms/step - loss: 0.2485 - avg_multilabel_BA_2: 0.8716 - val_loss: 0.2395 - val_avg_multilabel_BA_2: 0.8703
Epoch 12/40
199/199 [==============================] - 0s 1ms/step - loss: 0.2490 - avg_multilabel_BA_2: 0.8690 - val_loss: 0.2441 - val_avg_multilabel_BA_2: 0.8672
Epoch 13/40
199/199 [==============================] - 0s 1ms/step - loss: 0.2513 - avg_multilabel_BA_2: 0.8654 - val_loss: 0.2482 - val_avg_multilabel_BA_2: 0.8636
Epoch 14/40
199/199 [==============================] - 0s 1ms/step - loss: 0.2522 - avg_multilabel_BA_2: 0.8617 - val_loss: 0.2451 - val_avg_multilabel_BA_2: 0.8599
Epoch 15/40
199/199 [=

Epoch 17/40
199/199 [==============================] - 0s 914us/step - loss: 0.1981 - avg_multilabel_BA_2: 0.8371 - val_loss: 0.2002 - val_avg_multilabel_BA_2: 0.8368
Epoch 18/40
199/199 [==============================] - 0s 922us/step - loss: 0.1934 - avg_multilabel_BA_2: 0.8365 - val_loss: 0.1977 - val_avg_multilabel_BA_2: 0.8361
Epoch 19/40
199/199 [==============================] - 0s 928us/step - loss: 0.2015 - avg_multilabel_BA_2: 0.8357 - val_loss: 0.2410 - val_avg_multilabel_BA_2: 0.8354
Epoch 20/40
199/199 [==============================] - 0s 919us/step - loss: 0.2042 - avg_multilabel_BA_2: 0.8349 - val_loss: 0.2291 - val_avg_multilabel_BA_2: 0.8346
Epoch 21/40
199/199 [==============================] - 0s 926us/step - loss: 0.2104 - avg_multilabel_BA_2: 0.8342 - val_loss: 0.2061 - val_avg_multilabel_BA_2: 0.8338
Epoch 22/40
199/199 [==============================] - 0s 917us/step - loss: 0.1950 - avg_multilabel_BA_2: 0.8334 - val_loss: 0.1965 - val_avg_multilabel_BA_2: 0.833

Epoch 25/40
199/199 [==============================] - 0s 910us/step - loss: 0.2733 - avg_multilabel_BA_2: 0.7847 - val_loss: 0.2670 - val_avg_multilabel_BA_2: 0.7839
Epoch 26/40
199/199 [==============================] - 0s 906us/step - loss: 0.2723 - avg_multilabel_BA_2: 0.7831 - val_loss: 0.2673 - val_avg_multilabel_BA_2: 0.7823
Epoch 27/40
199/199 [==============================] - 0s 897us/step - loss: 0.2726 - avg_multilabel_BA_2: 0.7815 - val_loss: 0.2676 - val_avg_multilabel_BA_2: 0.7808
Epoch 28/40
199/199 [==============================] - 0s 904us/step - loss: 0.2728 - avg_multilabel_BA_2: 0.7800 - val_loss: 0.2678 - val_avg_multilabel_BA_2: 0.7792
Epoch 29/40
199/199 [==============================] - 0s 907us/step - loss: 0.2719 - avg_multilabel_BA_2: 0.7784 - val_loss: 0.2710 - val_avg_multilabel_BA_2: 0.7777
Epoch 30/40
199/199 [==============================] - 0s 916us/step - loss: 0.2736 - avg_multilabel_BA_2: 0.7769 - val_loss: 0.2677 - val_avg_multilabel_BA_2: 0.776

In [5]:
'''def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'''

'def nan_mse(y_actual, y_predicted):\n    stack = tf.stack((tf.math.is_nan(y_actual), \n                      tf.math.is_nan(y_predicted)),\n                     axis=1)\n    is_nans = tf.keras.backend.any(stack, axis=1)\n    per_instance = tf.where(is_nans,\n                            tf.zeros_like(y_actual),\n                            tf.square(tf.subtract(y_predicted, y_actual)))\n    print(stack)\n    print(per_instance)\n    return tf.reduce_mean(per_instance, axis=0)\n\nprint(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'

In [6]:
paths = ['/home/wander/OtherProjects/har_flower/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)

Trial 100 Complete [00h 00m 01s]
val_avg_multilabel_BA_2: 0.9182575345039368

Best val_avg_multilabel_BA_2 So Far: 0.918304443359375
Total elapsed time: 00h 09m 43s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
51/51 [==============================] - 1s 7ms/step - loss: 0.3554 - avg_multilabel_BA_2: 0.9182 - val_loss: 0.2405 - val_avg_multilabel_BA_2: 0.9182
Epoch 2/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1959 - avg_multilabel_BA_2: 0.9182 - val_loss: 0.1860 - val_avg_multilabel_BA_2: 0.9182
Epoch 3/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1543 - avg_multilabel_BA_2: 0.9182 - val_loss: 0.1562 - val_avg_multilabel_BA_2: 0.9182
Epoch 4/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1322 - avg_multilabel_BA_2: 0.9182 - val_loss: 0.1396 - val_avg_multilabel_BA_2: 0.9182
Epoch 5/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1197 - avg_multilabel_BA_2: 0.9182 - val_loss: 0.1293 - val_avg_multil

Epoch 50/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0512 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0709 - val_avg_multilabel_BA_2: 0.9185
Epoch 51/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0507 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.0696 - val_avg_multilabel_BA_2: 0.9185
Epoch 52/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0493 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.0704 - val_avg_multilabel_BA_2: 0.9185
Epoch 53/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0490 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.0705 - val_avg_multilabel_BA_2: 0.9185
Epoch 54/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0484 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.0701 - val_avg_multilabel_BA_2: 0.9185
Epoch 55/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0476 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.0673 - val_avg_multilabel_BA_2: 0.9185
Epoch 56/100
51/

Epoch 100/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0277 - avg_multilabel_BA_2: 0.9190 - val_loss: 0.0597 - val_avg_multilabel_BA_2: 0.9190
Best epoch: 100
Epoch 1/100
51/51 [==============================] - 1s 7ms/step - loss: 0.3613 - avg_multilabel_BA_2: 0.9189 - val_loss: 0.2529 - val_avg_multilabel_BA_2: 0.9189
Epoch 2/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1936 - avg_multilabel_BA_2: 0.9189 - val_loss: 0.1854 - val_avg_multilabel_BA_2: 0.9189
Epoch 3/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1496 - avg_multilabel_BA_2: 0.9189 - val_loss: 0.1573 - val_avg_multilabel_BA_2: 0.9189
Epoch 4/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1291 - avg_multilabel_BA_2: 0.9189 - val_loss: 0.1355 - val_avg_multilabel_BA_2: 0.9189
Epoch 5/100
51/51 [==============================] - 0s 1ms/step - loss: 0.1176 - avg_multilabel_BA_2: 0.9189 - val_loss: 0.1272 - val_avg_multilabel_BA_2: 0.9189
Epoc

Epoch 50/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0526 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0750 - val_avg_multilabel_BA_2: 0.9191
Epoch 51/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0522 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0721 - val_avg_multilabel_BA_2: 0.9191
Epoch 52/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0515 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0728 - val_avg_multilabel_BA_2: 0.9191
Epoch 53/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0520 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0727 - val_avg_multilabel_BA_2: 0.9191
Epoch 54/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0515 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0717 - val_avg_multilabel_BA_2: 0.9192
Epoch 55/100
51/51 [==============================] - 0s 1ms/step - loss: 0.0498 - avg_multilabel_BA_2: 0.9192 - val_loss: 0.0711 - val_avg_multilabel_BA_2: 0.9192
Epoch 56/100
51/

Epoch 100/100
25/25 [==============================] - 0s 646us/step - loss: 0.0753 - avg_multilabel_BA_2: 0.9196
Test results - Loss: 0.07531154155731201 - Averaged Balanced Accuracy: 0.9195777773857117%
Averaged Balanced Accuracy: 0.919579
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
40/40 [==============================] - 1s 9ms/step - loss: 0.3039 - avg_multilabel_BA_2: 0.9196 - val_loss: 0.2496 - val_avg_multilabel_BA_2: 0.9195
Epoch 2/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2402 - avg_multilabel_BA_2: 0.9195 - val_loss: 0.2241 - val_avg_multilabel_BA_2: 0.9195
Epoch 3/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2197 - avg_multilabel_BA_2: 0.9195 - val_loss: 0.2044 - val_avg_multilabel_BA_2: 0.9195
Epoch 4/100
40/40 [================

...

40/40 [==============================] - 0s 1ms/step - loss: 0.0665 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0948 - val_avg_multilabel_BA_2: 0.9191
Epoch 98/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0661 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0939 - val_avg_multilabel_BA_2: 0.9191
Epoch 99/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0651 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0989 - val_avg_multilabel_BA_2: 0.9191
Epoch 100/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0651 - avg_multilabel_BA_2: 0.9191 - val_loss: 0.0937 - val_avg_multilabel_BA_2: 0.9191
Best epoch: 1
20/20 [==============================] - 0s 645us/step - loss: 0.2691 - avg_multilabel_BA_2: 0.9190
Test results - Loss: 0.26914623379707336 - Averaged Balanced Accuracy: 0.9190047383308411%
Averaged Balanced Accuracy: 0.918989
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:te

40/40 [==============================] - 0s 1ms/step - loss: 0.1061 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1123 - val_avg_multilabel_BA_2: 0.9185
Epoch 44/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1049 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1124 - val_avg_multilabel_BA_2: 0.9185
Epoch 45/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1046 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1113 - val_avg_multilabel_BA_2: 0.9185
Epoch 46/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1030 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1131 - val_avg_multilabel_BA_2: 0.9185
Epoch 47/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1024 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1141 - val_avg_multilabel_BA_2: 0.9185
Epoch 48/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1030 - avg_multilabel_BA_2: 0.9185 - val_loss: 0.1118 - val_avg_multilabel_BA_2: 0.9185
Epoch 49/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.0730 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0971 - val_avg_multilabel_BA_2: 0.9184
Epoch 94/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0720 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0964 - val_avg_multilabel_BA_2: 0.9184
Epoch 95/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0704 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0963 - val_avg_multilabel_BA_2: 0.9184
Epoch 96/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0696 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0983 - val_avg_multilabel_BA_2: 0.9184
Epoch 97/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0692 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0968 - val_avg_multilabel_BA_2: 0.9184
Epoch 98/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0691 - avg_multilabel_BA_2: 0.9184 - val_loss: 0.0964 - val_avg_multilabel_BA_2: 0.9184
Epoch 99/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.1093 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1172 - val_avg_multilabel_BA_2: 0.9178
Epoch 40/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1100 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1150 - val_avg_multilabel_BA_2: 0.9178
Epoch 41/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1077 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1146 - val_avg_multilabel_BA_2: 0.9178
Epoch 42/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1069 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1145 - val_avg_multilabel_BA_2: 0.9178
Epoch 43/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1070 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1136 - val_avg_multilabel_BA_2: 0.9178
Epoch 44/100
40/40 [==============================] - 0s 1ms/step - loss: 0.1051 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1139 - val_avg_multilabel_BA_2: 0.9178
Epoch 45/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.0775 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.0993 - val_avg_multilabel_BA_2: 0.9177
Epoch 90/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0765 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.0979 - val_avg_multilabel_BA_2: 0.9177
Epoch 91/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0764 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1008 - val_avg_multilabel_BA_2: 0.9177
Epoch 92/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0758 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.0998 - val_avg_multilabel_BA_2: 0.9177
Epoch 93/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0770 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.0981 - val_avg_multilabel_BA_2: 0.9177
Epoch 94/100
40/40 [==============================] - 0s 1ms/step - loss: 0.0765 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.0985 - val_avg_multilabel_BA_2: 0.9177
Epoch 95/100
40/40 [=========

In [7]:

bas = []
config = {
        #'df_path': path,
        'df_path': '../full_data/exp_/fold_0/raw_40.csv',
        'neurons_1' : 32, 
        'neurons_2' : 16, 
        'labels': labels
}

har = utils.HAR(config)
test_results, ba = har.run()
#model, best_hps, best_epoch, test_results, ba = har.hypertunning()
bas.append(ba.numpy())


print(bas)
print(max(bas))

FileNotFoundError: [Errno 2] No such file or directory: '../full_data/exp_/fold_0/raw_40.csv'

In [8]:

import glob
bas = []
folderpath = '/home/wander/OtherProjects/har_flower/full_data'
for path in glob.iglob(f'{folderpath}/**.csv', recursive=True):
    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
    

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
50/50 [==============================] - 1s 8ms/step - loss: 0.3350 - avg_multilabel_BA_2: 0.9176 - val_loss: 0.2460 - val_avg_multilabel_BA_2: 0.9176
Epoch 2/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2056 - avg_multilabel_BA_2: 0.9176 - val_loss: 0.2111 - val_avg_multilabel_BA_2: 0.9176
Epoch 3/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1743 - avg_multilabel_BA_2: 0.9176 - val_loss: 0.1965 - val_avg_multilabel_BA_2: 0.9176
Epoch 4/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1593 - avg_multilabel_BA_2: 0.9176 - val_loss: 0.1878 - val_avg_multilabel_BA_2: 0.9176
Epoch 5/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1507 - avg_multilabel_BA_2: 0.9176 - va

50/50 [==============================] - 0s 2ms/step - loss: 0.0687 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1227 - val_avg_multilabel_BA_2: 0.9175
Epoch 50/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0678 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1252 - val_avg_multilabel_BA_2: 0.9175
Epoch 51/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0669 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1242 - val_avg_multilabel_BA_2: 0.9175
Epoch 52/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0659 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1235 - val_avg_multilabel_BA_2: 0.9175
Epoch 53/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0656 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1230 - val_avg_multilabel_BA_2: 0.9175
Epoch 54/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0647 - avg_multilabel_BA_2: 0.9175 - val_loss: 0.1286 - val_avg_multilabel_BA_2: 0.9175
Epoch 55/100
50/50 [=========

50/50 [==============================] - 0s 2ms/step - loss: 0.0458 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1228 - val_avg_multilabel_BA_2: 0.9178
Epoch 100/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0454 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.1296 - val_avg_multilabel_BA_2: 0.9178
Best epoch: 100
Epoch 1/100
50/50 [==============================] - 1s 8ms/step - loss: 0.3959 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.2783 - val_avg_multilabel_BA_2: 0.9178
Epoch 2/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2356 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.2369 - val_avg_multilabel_BA_2: 0.9178
Epoch 3/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1987 - avg_multilabel_BA_2: 0.9178 - val_loss: 0.2151 - val_avg_multilabel_BA_2: 0.9177
Epoch 4/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1768 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.2014 - val_avg_multilabel_BA_2: 0.9177
Epoch 5/100
50/5

Epoch 49/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0680 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1220 - val_avg_multilabel_BA_2: 0.9177
Epoch 50/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0667 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1174 - val_avg_multilabel_BA_2: 0.9177
Epoch 51/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0657 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1188 - val_avg_multilabel_BA_2: 0.9177
Epoch 52/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0659 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1156 - val_avg_multilabel_BA_2: 0.9177
Epoch 53/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0648 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1159 - val_avg_multilabel_BA_2: 0.9177
Epoch 54/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0636 - avg_multilabel_BA_2: 0.9177 - val_loss: 0.1162 - val_avg_multilabel_BA_2: 0.9177
Epoch 55/100
50/

Epoch 99/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0437 - avg_multilabel_BA_2: 0.9180 - val_loss: 0.1208 - val_avg_multilabel_BA_2: 0.9180
Epoch 100/100
25/25 [==============================] - 0s 654us/step - loss: 0.0901 - avg_multilabel_BA_2: 0.9180
Test results - Loss: 0.09005933254957199 - Averaged Balanced Accuracy: 0.9180314540863037%
Averaged Balanced Accuracy: 0.918033
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
40/40 [==============================] - 1s 9ms/step - loss: 0.3985 - avg_multilabel_BA_2: 0.9180 - val_loss: 0.2616 - val_avg_multilabel_BA_2: 0.9180
Epoch 2/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2522 - avg_multilabel_BA_2: 0.9180 - val_loss: 0.2398 - val_avg_multilabel_BA_2: 0.9180
Epoch 3/100
40/40 [===============

...

Epoch 97/100
98/98 [==============================] - 0s 989us/step - loss: 0.0847 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.1045 - val_avg_multilabel_BA_2: 0.9221
Epoch 98/100
98/98 [==============================] - 0s 1ms/step - loss: 0.0835 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.1064 - val_avg_multilabel_BA_2: 0.9221
Epoch 99/100
98/98 [==============================] - 0s 1ms/step - loss: 0.0837 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.1044 - val_avg_multilabel_BA_2: 0.9221
Epoch 100/100
98/98 [==============================] - 0s 997us/step - loss: 0.0826 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.1040 - val_avg_multilabel_BA_2: 0.9221
Best epoch: 1
48/48 [==============================] - 0s 596us/step - loss: 0.2473 - avg_multilabel_BA_2: 0.9221
Test results - Loss: 0.24734756350517273 - Averaged Balanced Accuracy: 0.9220572113990784%
Averaged Balanced Accuracy: 0.922050
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/or

21/21 [==============================] - 0s 2ms/step - loss: 0.0201 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0263 - val_avg_multilabel_BA_2: 0.9220
Epoch 44/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0195 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0254 - val_avg_multilabel_BA_2: 0.9220
Epoch 45/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0188 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0249 - val_avg_multilabel_BA_2: 0.9220
Epoch 46/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0180 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0244 - val_avg_multilabel_BA_2: 0.9220
Epoch 47/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0176 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0246 - val_avg_multilabel_BA_2: 0.9220
Epoch 48/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0172 - avg_multilabel_BA_2: 0.9220 - val_loss: 0.0236 - val_avg_multilabel_BA_2: 0.9220
Epoch 49/100
21/21 [=========

21/21 [==============================] - 0s 2ms/step - loss: 0.0069 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0178 - val_avg_multilabel_BA_2: 0.9221
Epoch 94/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0067 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0178 - val_avg_multilabel_BA_2: 0.9221
Epoch 95/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0066 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0179 - val_avg_multilabel_BA_2: 0.9221
Epoch 96/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0065 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0177 - val_avg_multilabel_BA_2: 0.9221
Epoch 97/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0065 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0178 - val_avg_multilabel_BA_2: 0.9221
Epoch 98/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0064 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0182 - val_avg_multilabel_BA_2: 0.9221
Epoch 99/100
21/21 [=========

Epoch 43/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0200 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0269 - val_avg_multilabel_BA_2: 0.9221
Epoch 44/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0195 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0266 - val_avg_multilabel_BA_2: 0.9221
Epoch 45/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0189 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0259 - val_avg_multilabel_BA_2: 0.9221
Epoch 46/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0184 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0253 - val_avg_multilabel_BA_2: 0.9221
Epoch 47/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0179 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0252 - val_avg_multilabel_BA_2: 0.9221
Epoch 48/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0174 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0244 - val_avg_multilabel_BA_2: 0.9221
Epoch 49/100
21/

Epoch 93/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0074 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0194 - val_avg_multilabel_BA_2: 0.9221
Epoch 94/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0073 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0189 - val_avg_multilabel_BA_2: 0.9221
Epoch 95/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0072 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0189 - val_avg_multilabel_BA_2: 0.9221
Epoch 96/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0071 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0191 - val_avg_multilabel_BA_2: 0.9221
Epoch 97/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0070 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0192 - val_avg_multilabel_BA_2: 0.9221
Epoch 98/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0069 - avg_multilabel_BA_2: 0.9221 - val_loss: 0.0197 - val_avg_multilabel_BA_2: 0.9221
Epoch 99/100
21/